## Process Outline
We need a set of manually categorized FERC plants with which to test the sklearn Classifier that we're making. We'll also use them to test the regression analysis that Alana is working on. What does this dataset look like, how do we create it?
* Pull FERC Plants table.
* Using a variety of search methods, to ensure that we get a wide variety of plants, identify sets of records in the FERC Form 1 Plants table that are comparable inter-year records.
* Searching can be done, for example, by matching a pattern against the plant name (which ought to show some cases in which the same plant is reported by different respondents, or where the ownership changes from year to year...), or by selecting all of the records from a given respondent, and identifying the records that pertain to a given plant consistently across years, even if the name changes.
* Diversity of plants is key -- we need to show a wide variety of different kinds of time series to the classifier so it knows how to deal with them.

In [87]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import init, mcoe, analysis, settings, outputs
import pudl.extract.ferc1
import pudl.transform.ferc1
import pudl.constants as pc

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [89]:
# For some reason these things don't stick if they're in the same cell as the
# %matplotlib inline call above, but if they're separate, they work fine.
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 56

In [90]:
ferc1_engine = pudl.extract.ferc1.connect_db()
ferc1_steam_raw = pd.read_sql('SELECT * FROM f1_steam;', ferc1_engine)
fc = pudl.transform.ferc1.FERCPlantClassifier()
fc._prepare_plants(ferc1_steam_raw)
ferc1_prep = fc._ferc1_steam

In [297]:
masks = [
    ((ferc1_prep.plant_name.str.match('.*cayuga.*')) &
     (ferc1_prep.plant_kind_cpi=='steam') &
     (ferc1_prep.yr_const=='1970')),

    ((ferc1_prep.plant_name.str.match('.*comanche.*')) &
     (ferc1_prep.respondent_id==148)),
    
    (ferc1_prep.plant_name.str.match('.*odemach.*2') &
     (ferc1_prep.yr_const=='1982')),
    
    (ferc1_prep.plant_name.str.match('.*klaunion.*') &
    (ferc1_prep.respondent_id==148)),
    
    (ferc1_prep.plant_name.str.match('.*klaunion.*') &
    (ferc1_prep.respondent_id==189)),
    
    (ferc1_prep.plant_name.str.match('.*klaunion.*') &
    (ferc1_prep.respondent_id==24)),
    
    (ferc1_prep.plant_name.str.match('.*effrey.*') &
    (ferc1_prep.respondent_id==191) &
    (ferc1_prep.expns_fuel > 0.0)),
    
    (ferc1_prep.plant_name.str.match('.*effrey.*') &
    (ferc1_prep.respondent_id==80) &
    (ferc1_prep.expns_fuel > 0.0)),
    
    (ferc1_prep.plant_name.str.match('.*effrey.*') &
    (ferc1_prep.respondent_id==182) &
    (ferc1_prep.expns_fuel > 0.0)),
    
    (ferc1_prep.plant_name.str.match('(la cygne 50%|lacygne 50%|lacygne 1.*)') &
    (ferc1_prep.respondent_id==79)),
    
    (ferc1_prep.plant_name.str.match('.*ygne.*2.*') &
    (ferc1_prep.respondent_id==79)),
    
    (ferc1_prep.plant_name.str.match('.*ygne.*#1.*') &
    (ferc1_prep.respondent_id==80)),
    
    (ferc1_prep.plant_name.str.match('.*ygne.*#2.*') &
    (ferc1_prep.respondent_id==80)),
    
    (ferc1_prep.respondent_id==182) &
    (ferc1_prep.plant_name=='sibley'),
    
    ferc1_prep.plant_name.str.match('.*neal.*1'),
    
    ferc1_prep.plant_name.str.match('.*neal.*2'),
    
    ((ferc1_prep.plant_name.str.match('.*neal.*3')) &
    (ferc1_prep.respondent_id==281)),
    
    ((ferc1_prep.plant_name.str.match('.*neal.*3')) &
    (ferc1_prep.respondent_id==210)),
    
    ((ferc1_prep.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
    (ferc1_prep.respondent_id==147)),
    
    ((ferc1_prep.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
    (ferc1_prep.respondent_id==176)),
    
    (ferc1_prep.plant_name.str.match('.*orktown.*')),
    
    (ferc1_prep.plant_name.str.match('chesterfield$')),
    
    ((ferc1_prep.plant_name.str.match('rockport total aeg')) &
                              (ferc1_prep.respondent_id==1)),
    
    ((ferc1_prep.plant_name.str.match('rockport total i&m')) &
                              (ferc1_prep.respondent_id==73)),
    
    ((ferc1_prep.plant_name.str.match('mount storm$')) &
                              (ferc1_prep.respondent_id==186)),
    
    ((ferc1_prep.plant_name.str.match('.*mitchell.*')) &
                              (ferc1_prep.respondent_id==127)),
    
    ((ferc1_prep.plant_name.str.match('.*mitchell.*share$')) &
                              (ferc1_prep.respondent_id==81)),
    
    ((ferc1_prep.plant_name.str.match('.*mitchell.*share$')) &
                              (ferc1_prep.respondent_id==452)),
    
    ((ferc1_prep.plant_name.str.match('.*mitchell.*share$')) &
                              (ferc1_prep.respondent_id==192)),
    
    (ferc1_prep.plant_name.str.match('.*irginia city.*')),
    
    ((ferc1_prep.plant_name.str.match('.*east bend.*')) &
            (ferc1_prep.respondent_id==178)),
    
    ((ferc1_prep.plant_name.str.match('.*east bend.*')) &
            (ferc1_prep.respondent_id==27)),
    
    ((ferc1_prep.plant_name.str.match('.*east bend.*')) &
            (ferc1_prep.respondent_id==42)),
    
    ((ferc1_prep.plant_name.str.match('.*arrison.*')) &
            (ferc1_prep.respondent_id==101)),
    
    ((ferc1_prep.plant_name.str.match('^amos-opco share$')) &
            (ferc1_prep.respondent_id==127)),
    
    ((ferc1_prep.plant_name.str.match('^amos-apco share$')) &
            (ferc1_prep.respondent_id==6)),
    
    ((ferc1_prep.plant_name.str.match('^amos$')) &
            (ferc1_prep.respondent_id==6)),
    
    ((ferc1_prep.plant_name.str.match('.*killen.*')) &
            (ferc1_prep.respondent_id==27)),
    
    ((ferc1_prep.plant_name.str.match('.*killen.*')) &
            (ferc1_prep.respondent_id==42) &
            (ferc1_prep.tot_capacity > 100.0)),
    
    ((ferc1_prep.plant_name.str.match('.*stuart.*')) &
            (ferc1_prep.respondent_id==42)),       
    
    ((ferc1_prep.plant_name.str.match('.*stuart.*')) &
            (ferc1_prep.respondent_id==31)),
    
    ((ferc1_prep.plant_name.str.match('.*stuart.*')) &
            (ferc1_prep.respondent_id==127)),
    
    ((ferc1_prep.plant_name.str.match('.*stuart.*')) &
            (ferc1_prep.respondent_id==452)),
    
    ((ferc1_prep.plant_name.str.match('comanche')) &
            (ferc1_prep.respondent_id==145)),
    
    ((ferc1_prep.plant_name.str.match('pawnee')) &
            (ferc1_prep.respondent_id==145)),
    
    ((ferc1_prep.plant_name.str.match('hayden')) &
            (ferc1_prep.respondent_id==145)),
    
    ((ferc1_prep.plant_name.str.match('hayden')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('craig')) &
            (ferc1_prep.respondent_id==145)),
    
    ((ferc1_prep.plant_name.str.match('craig')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*')) &
            (ferc1_prep.respondent_id==122)),
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*')) &
            (ferc1_prep.respondent_id==187)),
    # Colstrip 1 & 2 and Colstrip 3 & 4 are owned by the same utility and
    # reported together. When only searching for '.*colstrip.*', the dataframe didn't have
    # the same shape to contatentate to the rest of the plants so instead the search is
    # for '.*colstrip.*2$' and '.*colstrip.*4$'.
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*2$')) &
            (ferc1_prep.respondent_id==150)),
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*4$')) &
            (ferc1_prep.respondent_id==150)),
    
    ((ferc1_prep.plant_name.str.match('^hunter.*1$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^hunter.*2$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^hunter.*3$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^jim bridger$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^jim bridger$')) &
            (ferc1_prep.respondent_id==70)),
    
    ((ferc1_prep.plant_name.str.match('^huntington$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^meramec$')) &
            (ferc1_prep.respondent_id==177)),
    
    ((ferc1_prep.plant_name.str.match('^valmy$')) &
           (ferc1_prep.respondent_id==70)),
    
    ((ferc1_prep.plant_name.str.match('.*valmy.*')) &
           (ferc1_prep.respondent_id==157)),
    
    ((ferc1_prep.plant_name.str.match('^big stone$')) &
           (ferc1_prep.respondent_id==132)),
    
    ((ferc1_prep.plant_name.str.match('^big stone$')) &
           (ferc1_prep.respondent_id==122)),
    
    ((ferc1_prep.plant_name.str.match('^big stone$')) &
           (ferc1_prep.respondent_id==95)),
    
    ((ferc1_prep.plant_name.str.match('four corners')) &
            (ferc1_prep.respondent_id==49)),
    
    ((ferc1_prep.plant_name.str.match('four corners')) &
            (ferc1_prep.respondent_id==176)),
    
    ((ferc1_prep.plant_name.str.match('four corners')) &
            (ferc1_prep.respondent_id==161)),

    # From 2004 through 2006 respondent_id 7 splits the Four Corners plant into units
    # 1, 2, and 3, and units 4 and 5. The combined records for 1, 2, and 3 are used along
    # with units 1, 2, and 3, which appear on their own in later years. The combined records 
    # for 4 and 5 are used along with units 4 and 5, which also appear on their own in later years.
    
    ((ferc1_prep.plant_name.str.match('four corners 1')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7) &
            (ferc1_prep.yr_const=='1963')),
    
    ((ferc1_prep.plant_name.str.match('four corners 2')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7) &
            (ferc1_prep.yr_const=='1963')),
    
    ((ferc1_prep.plant_name.str.match('four corners 3')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7) &
            ((ferc1_prep.yr_const=='1963')|(ferc1_prep.yr_const=='1964'))),
    
    ((ferc1_prep.plant_name.str.match('four corners 4')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7)&
            (ferc1_prep.yr_const=='1969')),
    
    ((ferc1_prep.plant_name.str.match('four corners 5')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7)&
            (ferc1_prep.yr_const=='1969')),
    
    
    # Other plants:
    
    ((ferc1_prep.plant_name.str.match('.*killen.*')) &
            (ferc1_prep.respondent_id==42) &
            (ferc1_prep.tot_capacity < 100.0)),
    
    ((ferc1_prep.plant_name.str.match('.*brown.*')) &
            (ferc1_prep.respondent_id==163) &
            (ferc1_prep.yr_const=='1979')),
    
    ((ferc1_prep.plant_name.str.match('.*brown.*')) &
            (ferc1_prep.respondent_id==163) &
            (ferc1_prep.yr_const=='1991')),
    
    ((ferc1_prep.plant_name.str.match('.*brown.*')) &
            (ferc1_prep.respondent_id==163) &
            (ferc1_prep.yr_const=='2002')),
    
    ((ferc1_prep.plant_name.str.match('.*acadia.*')) &
    (ferc1_prep.respondent_id==87)),
    
    ((ferc1_prep.plant_name.str.match('.*acadia.*')) &
    (ferc1_prep.respondent_id==454)),
    
    ((ferc1_prep.plant_name.str.match('.*acadia.*')) &
    (ferc1_prep.respondent_id==22))
    
    
]

In [6]:
# In this cell, you can mess around with various selections from the ferc plant
# dataframe, until it's just getting the plants you want to keep, and then add
# your new mask to the list of masks above.
#new_mask = (
#    (ferc1_prep.respondent_id==210) &
#    (ferc1_prep.yr_const=='2003')
#)
#ferc1_prep[new_mask]

In [298]:
ferc1_train = pd.DataFrame({}, columns=range(2004,2017))
new_plants = [ferc1_train]
for mask in masks:
    new_plants = new_plants + [ferc1_prep[mask][['record_id','report_year']].\
                                 set_index('report_year').transpose()]
ferc1_train = pd.concat(new_plants).reset_index().drop('index', axis=1)
ferc1_train

report_year,2004,2004,2005,2005,2006,2006,2007,2007,2008,2008,2009,2009,2010,2010,2011,2011,2012,2012,2013,2013,2014,2014,2015,2015,2016,2016
0,2004_144_0_5,2004_144_0_5,2005_144_0_5,2005_144_0_5,2006_144_0_5,2006_144_0_5,2007_144_0_5,2007_144_0_5,2008_144_0_5,2008_144_0_5,2009_144_0_5,2009_144_0_5,2010_144_0_5,2010_144_0_5,2011_144_1_4,2011_144_1_4,2012_144_1_4,2012_144_1_4,2013_144_1_4,2013_144_1_4,2014_144_1_4,2014_144_1_4,2015_144_1_4,2015_144_1_4,2016_144_1_4,2016_144_1_4
1,2004_148_0_5,2004_148_0_5,2005_148_0_5,2005_148_0_5,2006_148_0_5,2006_148_0_5,2007_148_0_5,2007_148_0_5,2008_148_0_5,2008_148_0_5,2009_148_0_5,2009_148_0_5,2010_148_0_5,2010_148_0_5,2011_148_0_5,2011_148_0_5,2012_148_0_5,2012_148_0_5,2013_148_0_5,2013_148_0_5,2014_148_0_5,2014_148_0_5,2015_148_0_5,2015_148_0_5,2016_148_0_5,2016_148_0_5
2,2004_22_0_3,2004_22_0_3,2005_22_0_3,2005_22_0_3,2006_22_0_3,2006_22_0_3,2007_22_0_3,2007_22_0_3,2008_22_0_3,2008_22_0_3,2009_22_0_3,2009_22_0_3,2010_22_0_3,2010_22_0_3,2011_22_0_3,2011_22_0_3,2012_22_0_3,2012_22_0_3,2013_22_0_3,2013_22_0_3,2014_22_0_3,2014_22_0_3,2015_22_0_3,2015_22_0_3,2016_22_0_3,2016_22_0_3
3,2004_148_1_2,2004_148_1_2,2005_148_1_2,2005_148_1_2,2006_148_1_2,2006_148_1_2,2007_148_1_2,2007_148_1_2,2008_148_1_2,2008_148_1_2,2009_148_1_2,2009_148_1_2,2010_148_1_2,2010_148_1_2,2011_148_1_2,2011_148_1_2,2012_148_1_2,2012_148_1_2,2013_148_1_2,2013_148_1_2,2014_148_1_2,2014_148_1_2,2015_148_1_2,2015_148_1_2,2016_148_1_2,2016_148_1_2
4,2004_189_0_4,2004_189_0_4,2005_189_0_4,2005_189_0_4,2006_189_0_1,2006_189_0_1,2007_189_0_1,2007_189_0_1,2008_189_0_1,2008_189_0_1,2009_189_0_1,2009_189_0_1,2010_189_0_1,2010_189_0_1,2011_189_0_1,2011_189_0_1,2012_189_0_1,2012_189_0_1,2013_189_0_1,2013_189_0_1,2014_189_0_1,2014_189_0_1,2015_189_0_1,2015_189_0_1,2016_189_0_1,2016_189_0_1
5,2004_24_2_1,2004_24_2_1,2005_24_2_1,2005_24_2_1,2006_24_0_1,2006_24_0_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2004_191_1_4,2004_191_1_4,2005_191_1_2,2005_191_1_2,2006_191_1_4,2006_191_1_4,2007_191_1_4,2007_191_1_4,2008_191_1_4,2008_191_1_4,2009_191_1_4,2009_191_1_4,2010_191_1_4,2010_191_1_4,2011_191_1_4,2011_191_1_4,2012_191_1_4,2012_191_1_4,2013_191_1_3,2013_191_1_3,2014_191_1_3,2014_191_1_3,2015_191_1_3,2015_191_1_3,2016_191_1_3,2016_191_1_3
7,2004_80_1_2,2004_80_1_2,2005_80_1_2,2005_80_1_2,2006_80_1_2,2006_80_1_2,2007_80_1_2,2007_80_1_2,2008_80_1_2,2008_80_1_2,2009_80_1_2,2009_80_1_2,2010_80_1_1,2010_80_1_1,2011_80_1_1,2011_80_1_1,2012_80_1_1,2012_80_1_1,2013_80_0_5,2013_80_0_5,2014_80_0_5,2014_80_0_5,2015_80_0_5,2015_80_0_5,2016_80_0_5,2016_80_0_5
8,2004_182_0_3,2004_182_0_3,2005_182_0_3,2005_182_0_3,2006_182_0_3,2006_182_0_3,2007_182_0_3,2007_182_0_3,2008_182_0_3,2008_182_0_3,2009_182_0_3,2009_182_0_3,2010_182_0_3,2010_182_0_3,2011_182_0_3,2011_182_0_3,2012_182_0_3,2012_182_0_3,2013_182_0_3,2013_182_0_3,2014_182_0_3,2014_182_0_3,2015_182_0_3,2015_182_0_3,2016_182_0_3,2016_182_0_3
9,2004_79_0_4,2004_79_0_4,2005_79_0_4,2005_79_0_4,2006_79_0_4,2006_79_0_4,2007_79_0_4,2007_79_0_4,2008_79_0_4,2008_79_0_4,2009_79_2_1,2009_79_2_1,2010_79_2_3,2010_79_2_3,2011_79_2_3,2011_79_2_3,2012_79_2_3,2012_79_2_3,2013_79_2_3,2013_79_2_3,2014_79_2_3,2014_79_2_3,2015_79_2_3,2015_79_2_3,2016_79_2_3,2016_79_2_3


In [9]:
ferc1_train.to_csv('ferc1_plantid_training.csv', index=False)

This is the suggested mask code:

In [296]:
new_mask = (
    (ferc1_prep.respondent_id==210) &
    (ferc1_prep.yr_const=='2003')
)
ferc1_prep[new_mask]

,record_id,report_year,spplmnt_num,row_number,respondent_id,plant_name,plant_kind_cpi,yr_const,tot_capacity,index,plant_kind,type_const,yr_installed,peak_demand,plant_hours,plnt_capability,when_not_limited,when_limited,avg_num_of_emp,net_generation,cost_land,cost_structure,cost_equipment,cost_of_plant_to,cost_per_kw,expns_operations,expns_fuel,expns_coolants,expns_steam,expns_steam_othr,expns_transfer,expns_electric,expns_misc_power,expns_rents,expns_allowances,expns_engnr,expns_structures,expns_boiler,expns_plants,expns_misc_steam,tot_prdctn_expns,expns_kwh,asset_retire_cost
847,2004_210_2_4,2004,2,4,210,greater dm energy,natural_gas,2003,576.0,1560,Gas Turbine,Conventional,2004,0.0,1403.0,491.0,0.0,0.0,23.0,1.490240e+08,553176.0,19572446.0,300690690.0,320816312.0,556.9728,201250.0,11507131.0,0.0,0.0,0.0,0.0,287963.0,681782.0,0.0,0.0,93273.0,1680.0,0.0,212081.0,32015.0,13017175.0,0.0873,0.0
1843,2005_210_2_4,2005,2,4,210,greater dm energy,natural_gas,2003,576.0,3393,Gas Turbine,Conventional,2004,496.0,4854.0,491.0,0.0,0.0,22.0,6.790860e+08,570770.0,32565669.0,292056924.0,325193363.0,564.5718,120329.0,48691906.0,0.0,0.0,0.0,0.0,1491917.0,628073.0,0.0,0.0,74380.0,104319.0,0.0,1319846.0,113449.0,52544219.0,0.0774,0.0
2041,2006_210_2_4,2006,2,4,210,greater dm energy,natural_gas,2003,576.0,3763,Gas Turbine,Conventional,2004,497.0,4220.0,491.0,0.0,0.0,25.0,8.171150e+08,571843.0,32573421.0,292167275.0,325312539.0,564.7787,163345.0,49964608.0,0.0,0.0,0.0,0.0,1616110.0,567423.0,0.0,0.0,179289.0,139511.0,0.0,3145942.0,258169.0,56034397.0,0.0686,0.0
3034,2007_210_2_4,2007,2,4,210,greater dm energy,natural_gas,2003,576.0,5573,Gas Turbine,Conventional,2004,449.0,5571.0,491.0,0.0,0.0,25.0,1.101195e+09,572400.0,32612776.0,302454438.0,335639614.0,582.7077,318239.0,65203292.0,0.0,0.0,0.0,0.0,1907266.0,419166.0,0.0,0.0,30327.0,88850.0,0.0,12431268.0,167144.0,80565552.0,0.0732,0.0
4052,2008_210_2_4,2008,2,4,210,greater dm energy,natural_gas,2003,576.0,7488,Gas Turbine,Conventional,2004,476.0,5194.0,494.0,0.0,0.0,25.0,1.043125e+09,572400.0,32878467.0,304268560.0,337719427.0,586.3184,272594.0,68681228.0,0.0,0.0,0.0,0.0,2124373.0,464329.0,0.0,0.0,235456.0,171257.0,0.0,4515215.0,141670.0,76606122.0,0.0734,0.0
5925,2009_210_2_4,2009,2,4,210,greater dm energy,natural_gas,2003,576.0,11029,Gas Turbine,Conventional,2004,461.0,1848.0,493.0,0.0,0.0,25.0,3.383760e+08,1306678.0,33133772.0,305477434.0,339917884.0,590.1352,279651.0,15729058.0,0.0,0.0,0.0,0.0,1555808.0,491760.0,0.0,2478.0,231710.0,127134.0,0.0,2241644.0,21648.0,20680891.0,0.0611,0.0
6036,2010_210_2_4,2010,2,4,210,greater dm energy,natural_gas,2003,576.0,11198,Gas Turbine,Conventional,2004,477.0,2236.0,493.0,0.0,0.0,26.0,4.488820e+08,1306678.0,33140216.0,291580216.0,326027110.0,566.0193,249737.0,21461062.0,0.0,0.0,0.0,0.0,1661247.0,587998.0,0.0,1402.0,244848.0,166201.0,0.0,3240671.0,32560.0,27645726.0,0.0616,0.0
7059,2011_210_3_4,2011,3,4,210,greater dm energy,natural_gas,2003,576.0,13033,Gas Turbine,Conventional,2004,482.0,1308.0,493.0,0.0,0.0,26.0,2.577860e+08,1306678.0,33140216.0,291588228.0,326035122.0,566.0332,316501.0,12691533.0,0.0,0.0,0.0,0.0,1497983.0,601108.0,0.0,329.0,251470.0,225181.0,0.0,3350520.0,95603.0,19030228.0,0.0738,0.0
8084,2012_210_3_4,2012,3,4,210,greater dm energy,natural_gas,2003,576.0,14923,Gas Turbine,Conventional,2004,489.0,2572.0,496.0,0.0,0.0,26.0,5.419400e+08,1306678.0,33140216.0,292004682.0,326451576.0,566.7562,424367.0,15507003.0,0.0,0.0,0.0,0.0,1679085.0,980332.0,0.0,5.0,194642.0,753668.0,0.0,6196945.0,657000.0,26393047.0,0.0487,0.0
9103,2013_210_3_4,2013,3,4,210,greater dm energy,natural_gas,2003,576.0,16933,Gas Turbine,Conventional,2004,492.0,1113.0,496.0,0.0,0.0,26.0,2.152580e+08,1306678.0,34152841.0,292540575.0,328000094.0,569.4446,422186.0,12881424.0,0.0,0.0,0.0,0.0,1285066.0,1220424.0,0.0,72.0,291777.0,239378.0,0.0,1983349.0,318825.0,18642501.0,0.0866,0.0


## FERC Masks from CES work
These masks won't work directly because of different dataframe names and available columns and column names... but they should give you hints on how to go grab these same plants.

In [ ]:
ferc_masks['rodemacher'] = (ferc_df.plant_name.str.match('.*odemach.*2'))

ferc_masks['oklaunion_psok'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                (ferc_df.utility_id==148))
ferc_masks['oklaunion_aeptx_north'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                       (ferc_df.utility_id==189))
ferc_masks['oklaunion_aeptx_central'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                         (ferc_df.utility_id==24))

ferc_masks['jeffrey_westar'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                                (ferc_df.utility_id==191) &
                                (ferc_df.expns_fuel > 0.0))
ferc_masks['jeffrey_ksgeco'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                                (ferc_df.utility_id==80) &
                                (ferc_df.expns_fuel > 0.0))
ferc_masks['jeffrey_kcpl'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                              (ferc_df.utility_id==182) &
                              (ferc_df.expns_fuel > 0.0))

ferc_masks['lacygne_1_kcpl'] = (
    (ferc_df.plant_name.str.match('(La Cygne 50%|Lacygne 50%|Lacygne 1.*)')) &
    (ferc_df.utility_id==79)
)
ferc_masks['lacygne_2_kcpl'] = ((ferc_df.plant_name.str.match('.*ygne.*2.*')) &
                               (ferc_df.utility_id==79))
ferc_masks['lacygne_1_kge'] = ((ferc_df.plant_name.str.match('.*ygne.*#1.*')) &
                              (ferc_df.utility_id==80))
ferc_masks['lacygne_2_kge'] = ((ferc_df.plant_name.str.match('.*ygne.*#2.*')) &
                              (ferc_df.utility_id==80))

ferc_masks['sibley'] = ((ferc_df.utility_id==182) &
                        (ferc_df.plant_name=='Sibley'))

ferc_masks['george_neal_1'] = ((ferc_df.plant_name.str.match('.*Neal.*1')))
ferc_masks['george_neal_2'] = ((ferc_df.plant_name.str.match('.*Neal.*2')))
ferc_masks['george_neal_3_ipl'] = (
    (ferc_df.plant_name.str.match('.*Neal.*3') &
    (ferc_df.utility_id==281)))
ferc_masks['george_neal_3_midam'] = (
    (ferc_df.plant_name.str.match('.*Neal.*3') &
    (ferc_df.utility_id==210)))

ferc_masks['san_juan_psnm'] = ((ferc_df.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
                               (ferc_df.utility_id==147))
ferc_masks['san_juan_tepco'] = ((ferc_df.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
                                (ferc_df.utility_id==176))

ferc_masks['yorktown'] = (ferc_df.plant_name.str.match('.*orktown.*'))

ferc_masks['chesterfield'] = (ferc_df.plant_name.str.match('Chesterfield$'))

ferc_masks['rockport_aeg'] = ((ferc_df.plant_name.str.match('Rockport Total Aeg')) &
                              (ferc_df.utility_id==1))
ferc_masks['rockport_inmi'] = ((ferc_df.plant_name.str.match('Rockport Total I&M')) &
                               (ferc_df.utility_id==73))

ferc_masks['mt_storm'] = ((ferc_df.plant_name.str.match('Mount Storm$')) &
                          (ferc_df.utility_id==186))

###########################################################################
# Mitchell (West Virginia) plant. Originally owned by Ohio Power Company,
# then split between Kentucky Power Company, and AEP, subsequently Wheeling
ferc_masks['mitchell_opc'] = ((ferc_df.plant_name.str.match('.*[Mm]itchell.*')) &
                              (ferc_df.utility_id==127))
ferc_masks['mitchell_kpc'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==81))
ferc_masks['mitchell_aep'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==452))
ferc_masks['mitchell_wpc'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==192))

ferc_masks['virginia_city'] = ((ferc_df.plant_name.str.match('.*irginia [Cc]ity.*')))

###########################################################################
# East Bend -- Ownership of Duke's share seems to have moved from one
# subsidiary to another. Also, why is only generator_id=2 showing up in EIA?
# Seems like another case of some bad NA values.
ferc_masks['east_bend_dukeoh'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                  (ferc_df.utility_id==27))
ferc_masks['east_bend_daypl'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                 (ferc_df.utility_id==42))
ferc_masks['east_bend_dukeky'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                  (ferc_df.utility_id==178))

###########################################################################
# Harrison plant, owned by FirstEnergy, sold to subsidiary Mon Energy in
# 2013 -- at that point FERC reported capacity jumps to 2000MW. Only a 
# couple of complete years of cost data in EIA -- suffers from NA values.
# Last 2 years of FERC data are totally wacked -- neeeds to be cleaned up.
# Also, capacity reported doesn't really match up between EIA & FERC... we're
# missing a third unit, for which there's *no* whole year of data that's valid
# in EIA.  Gah.
ferc_masks['harrison'] = ((ferc_df.plant_name.str.match('.*arrison.*')) &
                                  (ferc_df.utility_id==101))

###########################################################################
# John Amos plant has 3 units. Jointly owned by Appalachian Power Co & 
# Ohio Power Co until 2013, after which point it is entirely App Pwr Co.
ferc_masks['amos_apco'] = ((ferc_df.plant_name.str.match('^Amos-Apco Share$')) &
                           (ferc_df.utility_id==6))
ferc_masks['amos_opco'] = ((ferc_df.plant_name.str.match('^Amos-Opco Share$')) &
                           (ferc_df.utility_id==127))
ferc_masks['amos_consolidated'] = ((ferc_df.plant_name.str.match('^Amos$')) &
                                   (ferc_df.utility_id==6))

###########################################################################
# Killen is slated for retirement mid-2018.  There's a weird dangling
# 12-18 MW generator of some kind being reported to FERC, which is why the
# DPL filter includes the nameplate capacity minimum of 100.0 MW. DPL also
# has fuel costs which are off by a factor of 1000x in 2015 only.
ferc_masks['killen_duke'] = ((ferc_df.plant_name.str.match('.*Killen.*')) &
                             (ferc_df.utility_id==27))
ferc_masks['killen_dpl'] = ((ferc_df.plant_name.str.match('.*Killen.*')) &
                              (ferc_df.utility_id==42) &
                              (ferc_df.nameplate_capacity_mw > 100.0))

###########################################################################
# J.M. Stuart plant has 4 coal fired units, and an ownership structure that
# has changed over the years. 2015 fuel cost numbers reported by DPL are off
# by a factor of 1000x.
ferc_masks['jm_stuart_duke'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                                (ferc_df.utility_id==27))
ferc_masks['jm_stuart_csp'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==31))
ferc_masks['jm_stuart_opco'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                                (ferc_df.utility_id==127))
ferc_masks['jm_stuart_aep'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==452))
ferc_masks['jm_stuart_dpl'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==42) &
                               (ferc_df.nameplate_capacity_mw > 100.0))

ferc_masks['comanche'] = ((ferc_df.plant_name.str.match('Comanche')) &
                          (ferc_df.utility_id==145))

ferc_masks['pawnee'] = ((ferc_df.plant_name.str.match('Pawnee')) &
                        (ferc_df.utility_id==145))

###########################################################################
# Hayden 1 & 2 -- operated by PSCO, partly owned by PacifiCorp
ferc_masks['hayden_psco'] = ((ferc_df.plant_name.str.match('Hayden')) &
                             (ferc_df.utility_id==145))
ferc_masks['hayden_pc'] = ((ferc_df.plant_name.str.match('Hayden')) &
                           (ferc_df.utility_id==134))

###########################################################################
# Craig station is a big ownership mess...
ferc_masks['craig_psco'] = ((ferc_df.plant_name.str.match('Craig')) &
                            (ferc_df.utility_id==145))
ferc_masks['craig_pc'] = ((ferc_df.plant_name.str.match('Craig')) &
                          (ferc_df.utility_id==134))

###########################################################################
# Colstrip is complicated. No EIA data for some reason -- cost are all
# unreported, but there should be other data, no? But it's all NA. I have
# included what I think the masks should be to capture those units anyway,
# in case we can fix it later or there's some real data there monthly...
# FERC ownership adds up to 1400MW in 2008 and later, but less earlier.
# Did the plant used to be largely owned by a public power entity?
ferc_masks['colstrip_nwc'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                              (ferc_df.utility_id==122)) # 240 MW 2004-2016
ferc_masks['colstrip_pc'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                             (ferc_df.utility_id==134)) # 155 MW 2004-2016
ferc_masks['colstrip_avista'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                                 (ferc_df.utility_id==187)) # 233 MW 2004-2016
ferc_masks['colstrip_pse'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                              (ferc_df.utility_id==150)) # 377 + 433 MW 2008-2016

###########################################################################
# Hunter plant. No EIA data coming through on 2014-2015 at annual freq.
# PacifiCorp is reporting the three units independently for some reason,
# which is great.
ferc_masks['hunter_1'] = ((ferc_df.plant_name.str.match('^Hunter.*1$')) &
                          (ferc_df.utility_id==134))
ferc_masks['hunter_2'] = ((ferc_df.plant_name.str.match('^Hunter.*2$')) &
                          (ferc_df.utility_id==134))
ferc_masks['hunter_3'] = ((ferc_df.plant_name.str.match('^Hunter.*3$')) &
                          (ferc_df.utility_id==134))

###########################################################################
# Jim Bridger -- 4 units, jointly owned by PacifiCorp & Idaho Power Co.
ferc_masks['jim_bridger_pc'] = ((ferc_df.plant_name.str.match('^Jim Bridger$')) &
                                (ferc_df.utility_id==134))
ferc_masks['jim_bridger_idpc'] = ((ferc_df.plant_name.str.match('^Jim Bridger$')) &
                                  (ferc_df.utility_id==70))

ferc_masks['huntington'] = ((ferc_df.plant_name.str.match('^Huntington$')) &
                            (ferc_df.utility_id==134))

###########################################################################
# Maramec - 4 units, one owner
ferc_masks['maramec'] = ((ferc_df.plant_name.str.match('^Meramec$')) &
                            (ferc_df.utility_id==177))

###########################################################################
# North Valmy - 2 units, jointly owned by Sierra Pacific & Idaho Power
ferc_masks['north_valmy_sp'] = ((ferc_df.plant_name.str.match('^Valmy$')) &
                            (ferc_df.utility_id==157))
ferc_masks['north_valmy_ip'] = ((ferc_df.plant_name.str.match('^Valmy$')) &
                            (ferc_df.utility_id==70))


###########################################################################
# Big Stone, three FERC respondents
ferc_masks['big_stone_ot'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==132))
ferc_masks['big_stone_nwc'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==122))
ferc_masks['big_stone_mdu'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==95))

###########################################################################
# Four Corners... 5 units. 5 owners. APS reports units after 2006
ferc_masks['four_courners_elp'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==49))
ferc_masks['four_courners_tep'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==176))
ferc_masks['four_courners_sce'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==161))

ferc_masks['four_courners_1_aps'] = (ferc_df.plant_name.str.match('Four Corners 1')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_2_aps'] = (ferc_df.plant_name.str.match('Four Corners 2')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_3_aps'] = (ferc_df.plant_name.str.match('Four Corners 3')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_4_aps'] = (ferc_df.plant_name.str.match('Four Corners 4')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_5_aps'] = (ferc_df.plant_name.str.match('Four Corners 5')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))

ferc_masks['four_courners_4_4c'] = (ferc_df.plant_name.str.match('Four Corners4')&
                                     (ferc_df.utility_id==147))
ferc_masks['four_courners_5_4c'] = (ferc_df.plant_name.str.match('Four Corners 5')&
                                     (ferc_df.utility_id==147))


ferc_masks['four_courners_1_psnm'] = (ferc_df.plant_name.str.match('Four Corners (1)')&
                                     (ferc_df.utility_id==513))

